In [ ]:
pip install stanza

In [ ]:
pip install stanza -U

In [1]:
import torch;
import stanza; stanza.download('en') # This downloads the English models for the neural pipelin
#nlp = stanza.Pipeline('en', download_method=None) # This sets up a default neural pipeline in English

2023-03-29 18:16:01 INFO: Downloading default packages for language: en (English) ...
2023-03-29 18:16:02 INFO: File exists: /Users/mikerich/stanza_resources/en/default.zip
2023-03-29 18:16:06 INFO: Finished downloading models and saved to /Users/mikerich/stanza_resources.


In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import state_union
from nltk.tokenize import PunktSentenceTokenizer
import glob
from tqdm import tqdm
from bs4 import BeautifulSoup
import spacy
from __future__ import unicode_literals
import spacy,en_core_web_sm
import textacy

In [3]:
nltk.download('words')
nltk.download('punkt')
nltk.download('maxent_ne_chunker')
nltk.download('averaged_perceptron_tagger')
nltk.download('state_union')

[nltk_data] Downloading package words to /Users/mikerich/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /Users/mikerich/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/mikerich/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/mikerich/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package state_union to
[nltk_data]     /Users/mikerich/nltk_data...
[nltk_data]   Package state_union is already up-to-date!


True

### One sentence test

In [ ]:
sentence = "Seller produces hot stamping foil which conforms and meets the Specification Requirements submitted, accepted and in Seller's possession for the Specification numbers listed attached in the Exhibit A., hereafter referred nto as 'Products'."

doc = nlp(sentence)

subj_verb_linkages = []

for word in doc.sentences[0].words:    # for each word in the sentence
    if word.deprel == 'root':          # if a word is the root, its dependency relation label is 'root'. thus if this is true, the curr word = root word
        
        root_verb = word.text          # save the root verb  
        root_id = word.id              # get the words id 
        
        for w in doc.sentences[0].words:                      # loop over words in the sentence
            if w.head == root_id and w.deprel == 'nsubj':     # if words head attribute = root_id, then its a a direct dependent of the root (is a child of the root)
                
                subject = w.text                              # save the associated subject
        

subj_verb_linkages = [subject, root_verb]



print(f"Root Verb: {root_verb}, Subject: {subject}")

subj_verb_linkages


### New ner() function

In [4]:
def ner(sentence): 
    
    entities = []
    verbs = []
    subjects = []
    subj_verb_linkages = []
    
    #Find the entities in the sentence
    words  = nltk.word_tokenize(sentence)        # break down the sentence into words
    tagged = nltk.pos_tag(words)                 # tag the words with Part of Speech 
    chunks = nltk.ne_chunk(tagged, binary=False) # binary = False named entities are classified (i.e PERSON, ORGANIZATION)
    
    for chunk in chunks:
        if hasattr(chunk, 'label'):              # hasattr(obj, key) -- checking if chunks have a label or not 
            entities.append(' '.join(c[0] for c in chunk)) # append entities to array
    
    
    #Find the verbs/subjects in the sentence
    nlp = spacy.load("en_core_web_sm")           # load in the spacy model
    doc = nlp(sentence)                          # create spacy doc object
    
    verbs = [token.text for token in doc if token.pos_ == "VERB"]     # traverse thru the tokens, find the verbs
    subjects = [token.text for token in doc if token.dep_ == "nsubj"]  # traverse thru the tokens, find the subjects
    
    
    #Find the Root Subject-verb linkages in the sentences using stanza
    nlp = stanza.Pipeline('en', download_method=None)            # this sets up a default neural pipeline in English
    doc = nlp(sentence)
     
    for word in doc.sentences[0].words:    # for each word in the sentence
        if word.deprel == 'root':          # if a word is the root, its dependency relation label is 'root'. thus if this is true, the curr word = root word

            root_verb = word.text          # save the root verb  
            root_id = word.id              # get the words id 
            
            for w in doc.sentences[0].words:                      # loop over words in the sentence
                if w.head == root_id and w.deprel == 'nsubj':     # if words head attribute = root_id, then its a a direct dependent of the root (is a child of the root)
                    subject = w.text 
                
    subj_verb_linkages = [subject, root_verb]   # subj_verb linkages array 
    
        
    return {'entities':entities, 
            'verbs':verbs,
            'subjects':subjects,
            'subj_verb_linkages':subj_verb_linkages} 

In [5]:
sentence = "Seller produces hot stamping foil which conforms and meets the Specification Requirements submitted, accepted and in Seller's possession for the Specification numbers listed attached in the Exhibit A., hereafter referred nto as 'Products'."

temp = ner(sentence)

2023-03-29 18:16:09 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| constituency | wsj       |
| depparse     | combined  |
| sentiment    | sstplus   |
| ner          | ontonotes |

2023-03-29 18:16:09 INFO: Using device: cpu
2023-03-29 18:16:09 INFO: Loading: tokenize
2023-03-29 18:16:09 INFO: Loading: pos
2023-03-29 18:16:09 INFO: Loading: lemma
2023-03-29 18:16:09 INFO: Loading: constituency
2023-03-29 18:16:09 INFO: Loading: depparse
2023-03-29 18:16:10 INFO: Loading: sentiment
2023-03-29 18:16:10 INFO: Loading: ner
2023-03-29 18:16:11 INFO: Done loading processors!


In [6]:
temp

{'entities': ['Seller', 'Requirements', 'Seller', 'Exhibit'],
 'verbs': ['produces',
  'stamping',
  'conforms',
  'meets',
  'submitted',
  'accepted',
  'listed',
  'attached',
  'referred'],
 'subjects': ['Seller', 'which', 'hereafter'],
 'subj_verb_linkages': ['Seller', 'produces']}